In [1]:
import pandas as pd

In [2]:
pop_df = pd.read_parquet("../00_data/population/population.parquet")

In [3]:
pop_df = pop_df[(pop_df["Year"] > 2002) & (pop_df["Year"] < 2016)]
pop_df.head()

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population
9309,01001,3,6,Alabama,Autauga County,2003,46800
9310,01003,3,6,Alabama,Baldwin County,2003,151509
9311,01005,3,6,Alabama,Barbour County,2003,28594
9312,01007,3,6,Alabama,Bibb County,2003,21399
9313,01009,3,6,Alabama,Blount County,2003,53457


In [4]:
mort_df = pd.read_csv("../00_data/all_death_causes_2003_2015.csv")
mort_df = mort_df[mort_df["State"] != "AK"]
mort_df = mort_df[mort_df["State"] != "CT"]
mort_df.head()

,County,State,FIPS_CODE,Year,Deaths
0,Baldwin County,AL,1003,2003,10.0
1,Jefferson County,AL,1073,2003,37.0
2,Jefferson County,AL,1073,2003,32.0
3,Mobile County,AL,1097,2003,26.0
6,Cochise County,AZ,4003,2003,11.0


In [5]:
print(pop_df.shape)
print(mort_df.shape)

(37236, 7)
(8819, 5)


In [6]:
pop_df["Year"].value_counts()

Year
2003    3103
2004    3103
2005    3103
2006    3103
2007    3103
2008    3103
2010    3103
2011    3103
2012    3103
2013    3103
2014    3103
2015    3103
Name: count, dtype: int64

In [7]:
mort_df["Year"].value_counts()

Year
2015    871
2014    818
2013    770
2011    759
2012    738
2010    714
2008    653
2009    651
2007    641
2006    623
2005    572
2004    530
2003    479
Name: count, dtype: int64

In [10]:
mort_df["FIPS_CODE"] = mort_df["FIPS_CODE"].astype(str)
pop_df["FIPS_CODE"] = pop_df["FIPS_CODE"].astype(str)

merged_df = pd.merge(
    pop_df,
    mort_df,
    how="left",
    on=["FIPS_CODE", "Year"],
    indicator=True,
)
print(merged_df.shape)
print(merged_df["_merge"].value_counts())
merged_df = merged_df.iloc[:, :7]
merged_df.head()

# so those that are left only imply they are rows that are missing mortality data

(37958, 11)
_merge
left_only     30926
both           7032
right_only        0
Name: count, dtype: int64


,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population
0,01001,3,6,Alabama,Autauga County,2003,46800
1,01003,3,6,Alabama,Baldwin County,2003,151509
2,01005,3,6,Alabama,Barbour County,2003,28594
3,01007,3,6,Alabama,Bibb County,2003,21399
4,01009,3,6,Alabama,Blount County,2003,53457
